**EDA: Netflix Movies and TV Shows. Part 1**

**Data source:** [https://www.kaggle.com/datasets/shivamb/netflix-shows/data](https://www.kaggle.com/datasets/shivamb/netflix-shows/data)

In [1]:
# Import necessary libraries for data cleaning
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
df = pd.read_csv('netflix_raw_data.csv')

**1. Initial exploration. Check data types and format of the dataset**

In [3]:

# Display summary information
df.info()

# Display the first two rows to inspect the dataset
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


**2. Data preprosessing**

In [4]:

# Check for duplicate rows. Return the count of duplicate rows
df.duplicated().sum()

0

In [5]:
# Check for missing values in each column
missing_data = df.isnull().sum()
missing_data

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [6]:
# Calculate the percentage of missing data for each column
missing_data_percentage = (df.isnull().mean() * 100).round(2)
missing_data_percentage

show_id          0.00
type             0.00
title            0.00
director        29.91
cast             9.37
country          9.44
date_added       0.11
release_year     0.00
rating           0.05
duration         0.03
listed_in        0.00
description      0.00
dtype: float64

In [7]:
# Handling missing data for further analysis

# Replace NaN values with 'Unknown' for specific columns
df['director'] = df['director'].fillna('Unknown') 
df['cast'] = df['cast'].fillna('Unknown') 
df['country'] = df['country'].fillna('Unknown')
df['rating'] = df['rating'].fillna('Unknown')

# Replace missing duration with '0 min'
df['duration'] = df['duration'].fillna('0 min')

# Forward fill missing 'date_added' values  
df['date_added'] = df['date_added'].ffill() 

In [8]:
# Remove spaces at the beginning and end of string columns
df = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
spaces_pattern = r'^\s|\s$'
has_extra_spaces_per_column = df.apply(lambda x: x.str.contains(spaces_pattern) if x.dtype == 'object' else False)

# Check the result of removing
column_count_extra_spaces = has_extra_spaces_per_column.sum()
column_count_extra_spaces

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [9]:
# Convert the 'date_added' column to datetime format
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df['date_added'].isna().sum()

# Check and count the number of missing values (NaT)
df['date_added'].isna().sum()

0

In [10]:
# Extract the year and month from the 'date_added' column
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

**3. Detect and handle data anomalies**

In [11]:
# Check for future years in the 'year_added' column
current_year = pd.Timestamp.today().year
future_years = df[df['year_added'] > current_year]

# Check for excessively old dates
old_dates = df[df['year_added'] < 1900]
df['year_added'] < 1900

future_years, old_dates


(Empty DataFrame
 Columns: [show_id, type, title, director, cast, country, date_added, release_year, rating, duration, listed_in, description, year_added, month_added]
 Index: [],
 Empty DataFrame
 Columns: [show_id, type, title, director, cast, country, date_added, release_year, rating, duration, listed_in, description, year_added, month_added]
 Index: [])

In [12]:
# Check 'rating' column values
df['rating'].value_counts()

rating
TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
Unknown        4
NC-17          3
UR             3
74 min         1
84 min         1
66 min         1
Name: count, dtype: int64

In [13]:
# Find 'min' values incorrectly placed in 'rating' from 'duration'
df[df['rating'].str.contains('min', na=False)]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
5541,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,2017-04-04,2017,74 min,0 min,Movies,"Louis C.K. muses on religion, eternal love, gi...",2017,4
5794,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,2016-09-16,2010,84 min,0 min,Movies,Emmy-winning comedy writer Louis C.K. brings h...,2016,9
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,2016-08-15,2015,66 min,0 min,Movies,The comic puts his trademark hilarious/thought...,2016,8


In [14]:
# Move 'min' values to 'duration'
df.loc[df['rating'].str.contains('min', na=False), 'duration'] = df['rating'] 

# Set 'rating' to 'Unknown' where 'min' values were moved
df.loc[df['rating'].str.contains('min', na=False), 'rating'] = 'Unknown'  

# Check the result
df[df['show_id'].str.contains('s5542|s5795|s5814', na=False)]


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
5541,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,2017-04-04,2017,Unknown,74 min,Movies,"Louis C.K. muses on religion, eternal love, gi...",2017,4
5794,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,2016-09-16,2010,Unknown,84 min,Movies,Emmy-winning comedy writer Louis C.K. brings h...,2016,9
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,2016-08-15,2015,Unknown,66 min,Movies,The comic puts his trademark hilarious/thought...,2016,8


In [15]:
# Extract movie duration in minutes
df['duration_minutes'] = df['duration'].str.extract(r'(\d+) min').astype(float)

# Extract number of seasons for series
df['duration_seasons'] = df['duration'].str.extract(r'(\d+) Seasons?').astype(float)

# Check the result
df[['type','duration', 'duration_seasons', 'duration_minutes']].head(7)



,type,duration,duration_seasons,duration_minutes
0,Movie,90 min,NaN,90.0
1,TV Show,2 Seasons,2.0,NaN
2,TV Show,1 Season,1.0,NaN
3,TV Show,1 Season,1.0,NaN
4,TV Show,2 Seasons,2.0,NaN
5,TV Show,1 Season,1.0,NaN
6,Movie,91 min,NaN,91.0


In [16]:
# Check for abnormal duration values

 # Duration over 4 hours
df[df['duration_minutes'] > 250].head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,duration_minutes,duration_seasons
717,s718,Movie,Headspace: Unwind Your Mind,Unknown,"Andy Puddicombe, Evelyn Lewis Prieto, Ginger D...",Unknown,2021-06-15,2021,TV-G,273 min,Documentaries,"Do you want to relax, meditate or sleep deeply...",2021,6,273.0,NaN
2491,s2492,Movie,The School of Mischief,Houssam El-Din Mustafa,"Suhair El-Babili, Adel Emam, Saeed Saleh, Youn...",Egypt,2020-05-21,1973,TV-14,253 min,"Comedies, Dramas, International Movies",A high school teacher volunteers to transform ...,2020,5,253.0,NaN
4253,s4254,Movie,Black Mirror: Bandersnatch,Unknown,"Fionn Whitehead, Will Poulter, Craig Parkinson...",United States,2018-12-28,2018,TV-MA,312 min,"Dramas, International Movies, Sci-Fi & Fantasy","In 1984, a young programmer begins to question...",2018,12,312.0,NaN


In [17]:
# Duration less than a minute
df[df['duration_minutes'] < 1].head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,duration_minutes,duration_seasons


Handle durations over 4 hours. Research on imdb.com shows:

- 'The School of Mischief' is 115 minutes
- 'Black Mirror: Bandersnatch' is 90 minutes
- 'Headspace: Unwind Your Mind' is an interactive show with each episode lasting 15 minutes.


In [18]:
# Replace incorrect durations with correct values
df.loc[df['show_id'] == 's718', 'duration_minutes'] = 15  # Correct duration for 'Headspace: Unwind Your Mind'
df.loc[df['show_id'] == 's2492', 'duration_minutes'] = 115  # Correct duration for 'The School of Mischief'
df.loc[df['show_id'] == 's4254', 'duration_minutes'] = 90  # Correct duration for 'Black Mirror: Bandersnatch'

# Check if the replacements were successful
df[df['show_id'].isin(['s718', 's2492', 's4254'])][['show_id', 'duration_minutes']]


,show_id,duration_minutes
717,s718,15.0
2491,s2492,115.0
4253,s4254,90.0


In [19]:
# Verify if there are still any issues in numeric data
df.describe()


,date_added,release_year,year_added,month_added,duration_minutes,duration_seasons
count,8807,8807.000000,8807.000000,8807.000000,6131.000000,2676.000000
mean,2019-05-16 14:22:10.123765248,2014.180198,2018.870217,6.653685,99.464198,1.764948
min,2008-01-01 00:00:00,1925.000000,2008.000000,1.000000,3.000000,1.000000
25%,2018-04-04 00:00:00,2013.000000,2018.000000,4.000000,87.000000,1.000000
50%,2019-07-01 00:00:00,2017.000000,2019.000000,7.000000,98.000000,1.000000
75%,2020-08-18 00:00:00,2019.000000,2020.000000,10.000000,114.000000,2.000000
max,2021-09-25 00:00:00,2021.000000,2021.000000,12.000000,237.000000,17.000000
std,NaN,8.819312,1.574361,3.436120,28.024933,1.582752


In [20]:
# Save the result
df.to_csv('netflix_clean_data.csv', index=False)